<a href="https://colab.research.google.com/github/CodingAW/Project-140/blob/main/Project_138.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle


In [2]:
from google.colab import files
u = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop

 61% 5.00M/8.20M [00:00<00:00, 32.6MB/s]
100% 8.20M/8.20M [00:00<00:00, 40.2MB/s]


In [6]:
!unzip articles-sharing-reading-from-cit-deskdrop.zip

Archive:  articles-sharing-reading-from-cit-deskdrop.zip
  inflating: shared_articles.csv     
  inflating: users_interactions.csv  


In [15]:
import pandas as pd
import numpy as py
import plotly.express as px
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
df1 = pd.read_csv("shared_articles.csv")
df2 = pd.read_csv("users_interactions.csv")

df1.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [9]:
df2.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


In [10]:
# Project 139

df1 = df1[df1['eventType'] == 'CONTENT SHARED']
df1.head(10)
  

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en
6,1459194557,CONTENT SHARED,-2148899391355011268,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.newsbtc.com/2016/03/28/banks-need-c...,Banks Need To Collaborate With Bitcoin and Fin...,It will take time until banks come around to t...,en
7,1459194599,CONTENT SHARED,4119190424078847945,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/blockchai...,Blockchain Technology Could Put Bank Auditors ...,When most people think about computers and rob...,en
8,1459194751,CONTENT SHARED,-7926018713416777892,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://news.bitcoin.com/conglomerates-intervi...,Why Decentralized Conglomerates Will Scale Bet...,"Bitcoin.com spoke with the OpenLedger CEO, Ron...",en
9,1459194842,CONTENT SHARED,3353902017498793780,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://www.cryptocoinsnews.com/ethereum-rise-...,The Rise And Growth of Ethereum Gets Mainstrea...,"Ethereum, considered by many to be the most pr...",en
10,1459210504,CONTENT SHARED,-9157338616628196758,5206835909720479405,-7864441319395545950,NaN,NaN,NaN,HTML,http://economia.ig.com.br/2016-03-27/situacao-...,Situação financeira ruim de varejistas pressio...,A queda nas vendas e a deterioração na situaçã...,pt


In [11]:
df2.shape

(72312, 8)

In [12]:
def find_total_events(df1_row):
  totalLikes = df2[(df2['contentId'] == df1_row['contentId']) & (df2['eventType'] == 'LIKE')].shape[0]
  totalViews = df2[(df2['contentId'] == df1_row['contentId']) & (df2['eventType'] == 'VIEW')].shape[0]
  totalBookmarks = df2[(df2['contentId'] == df1_row['contentId']) & (df2['eventType'] == 'BOOKMARK')].shape[0]
  totalFollows = df2[(df2['contentId'] == df1_row['contentId']) & (df2['eventType'] == 'FOLLOW')].shape[0]
  totalComments = df2[(df2['contentId'] == df1_row['contentId']) & (df2['eventType'] == 'COMMENT CREATED')].shape[0]

  return totalLikes + totalViews + totalBookmarks + totalFollows + totalComments

df1['totalEvents'] = df1.apply(find_total_events, axis=1)
df1.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang,totalEvents
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en,1
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en,1
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en,13
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en,0
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en,2


In [13]:
df1 = df1.sort_values("totalEvents", ascending= False)
df1.head(10)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang,totalEvents
3096,1487246811,CONTENT SHARED,-4029704725707465084,6013226412048763966,-6569695881431984742,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,HTML,http://www.cnbc.com/2016/12/21/former-google-c...,Former Google career coach shares a visual tri...,"If you want 2017 to be an exciting year, desig...",en,433
1671,1467813728,CONTENT SHARED,-133139342397538859,4918484843075254252,-5701227433817087697,NaN,NaN,NaN,HTML,http://gq.globo.com/Prazeres/Poder/Carreira/no...,"Novo workaholic trabalha, pratica esportes e t...",Novo workaholic não abre mão do esporte e da f...,pt,315
1814,1468867647,CONTENT SHARED,-6783772548752091658,4918484843075254252,-8995217520473210153,NaN,NaN,NaN,HTML,http://www.caroli.org/livro-retrospectivas-div...,Livro: Retrospectivas Divertidas,"Neste livro, nós fornecemos um conjunto de fer...",pt,294
1317,1465484901,CONTENT SHARED,8657408509986329668,-8020832670974472349,838596071610016700,NaN,NaN,NaN,HTML,https://medium.com/practical-blend/pull-reques...,Pull request first - Practical Blend,Pull request first After two years of working ...,en,294
588,1461629452,CONTENT SHARED,-6843047699859121724,7527226129639571966,-1297230017812472163,NaN,NaN,NaN,HTML,https://medium.com/@jeffersoncn/ganhe-6-meses-...,"Ganhe 6 meses de acesso ao Pluralsight, maior ...","Ganhe 6 meses de acesso ao Pluralsight, maior ...",pt,281
2308,1473593226,CONTENT SHARED,-2358756719610361882,-9120685872592674274,5016857076790212194,NaN,NaN,NaN,HTML,http://www.attps.com.br/cinco-motivos-para-inv...,Custo do Erro - Cinco motivos para investir em...,"Atualmente, o custo de manutenção de software ...",pt,280
1902,1469678235,CONTENT SHARED,-8208801367848627943,-3390049372067052505,2045534933671019150,NaN,NaN,NaN,HTML,http://www.geekwire.com/2016/ray-kurzweil-worl...,Ray Kurzweil: The world isn't getting worse - ...,"Ray Kurzweil, the author, inventor, computer s...",en,266
1942,1469992520,CONTENT SHARED,2581138407738454418,6756039155228175109,611377856408166257,NaN,NaN,NaN,HTML,https://medium.com/@rdsubhas/10-modern-softwar...,10 Modern Software Over-Engineering Mistakes,10 Modern Software Over-Engineering Mistakes F...,en,255
1724,1468247818,CONTENT SHARED,-1297580205670251233,534764222466712491,-9077035980881675856,NaN,NaN,NaN,HTML,https://viagensdealline.com/2013/08/20/a-minha...,A minha viagem à Maternidade #tetodomundo,"Já fazia uma semana, desde o dia 26 de dezembr...",pt,253
2095,1471520397,CONTENT SHARED,-1633984990770981161,2195040187466632600,1000828687449358272,NaN,NaN,NaN,HTML,https://medium.com/@roxrogge/ux-ou-ui-4c0a1bcb...,UX ou UI?,UX ou UI? Tenho escutado essa pergunta com fre...,pt,249


In [20]:
# Project 140

def clean_data(x):
  if isinstance(x, str):
    return x.lower()
  else:
    return ""

df1['title'] = df1['title'].apply(clean_data)

df1.head(3)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang,totalEvents
3096,1487246811,CONTENT SHARED,-4029704725707465084,6013226412048763966,-6569695881431984742,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,HTML,http://www.cnbc.com/2016/12/21/former-google-c...,former google career coach shares a visual tri...,"If you want 2017 to be an exciting year, desig...",en,433
1671,1467813728,CONTENT SHARED,-133139342397538859,4918484843075254252,-5701227433817087697,NaN,NaN,NaN,HTML,http://gq.globo.com/Prazeres/Poder/Carreira/no...,"novo workaholic trabalha, pratica esportes e t...",Novo workaholic não abre mão do esporte e da f...,pt,315
1814,1468867647,CONTENT SHARED,-6783772548752091658,4918484843075254252,-8995217520473210153,NaN,NaN,NaN,HTML,http://www.caroli.org/livro-retrospectivas-div...,livro: retrospectivas divertidas,"Neste livro, nós fornecemos um conjunto de fer...",pt,294


In [28]:
titles = df1['title']
print(titles[1])

novo workaholic trabalha, pratica esportes e tem tempo para a família. conheça


In [22]:
count = CountVectorizer(stop_words= "english")
count_matrix = count.fit_transform(df1['title'])

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
df1 = df1.reset_index()
indices = pd.Series(df1.index, index= df1['title'])

In [29]:
def get_reccommendations(title, cosine_sim):
  idx = indices[title]
  sim_scores = list(enumerate(cosine_sim[idx]))

  sim_scores = sorted(sim_scores, key= lambda x: x[1], reverse= True)
  sim_scores = sim_scores[1:11]
  movie_indices = [i[0] for  i in sim_scores]
  return df1['title'].iloc[movie_indices]

get_reccommendations("novo workaholic trabalha, pratica esportes e tem tempo para a família. conheça", cosine_sim2)

649     para rebater estatuto da família, campanha mud...
1066           lista tem 15 dicas para google docs; teste
1565    conheça o google duo: um app de chamadas de ví...
1577    "5g está para iot como 4g esteve para o smartp...
625     pabllo vittar reina no novo comercial da skol ...
78                                     criadas para casar
2174    webinar do aem para ajuda na preparação para f...
2378    conheça as ferramentas digitais que utilizo na...
1400    3 noções básicas essenciais para a criação de ...
386     globo g1 lança bot para o telegram que te ajud...
Name: title, dtype: object